In [200]:
from qiskit import QuantumCircuit
import numpy as np
from qiskit_aer import StatevectorSimulator
from qiskit_aer.primitives import SamplerV2
from qbench.schemes.postselection import benchmark_certification_using_postselection
from qbench.schemes.direct_sum import benchmark_certification_using_direct_sum

In [201]:
### certification experiment for measurement in Hadamard basis using postselection and direct sum ###

def state_prep():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0,1)
    return circuit.to_instruction()
    
def u_dag():
    circuit = QuantumCircuit(1)
    circuit.h(0)
    return circuit.to_instruction()

def v0_dag():
    circuit = QuantumCircuit(1)
    circuit.ry(2 * np.arcsin(np.sqrt(0.05)), 0)
    
    return circuit.to_instruction()

def v1_dag():
    circuit = QuantumCircuit(1)
    circuit.ry(2 * np.arcsin(np.sqrt(0.05)), 0)
    circuit.x(0)
    return circuit.to_instruction()

def v0_v1_direct_sum_dag():
    circuit = QuantumCircuit(2)
    circuit.p(-np.pi, 0)
    circuit.ry(-2 * np.arcsin(np.sqrt(0.05)), 0)
    circuit.cx(0, 1)
    return circuit.to_instruction()

In [202]:
simulator = StatevectorSimulator()

In [203]:
postselection_result = benchmark_certification_using_postselection(
    backend=simulator,
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_dag=v0_dag(),
    v1_dag=v1_dag(),
    num_shots_per_measurement=10
)

postselection_result

0.7142857142857143

In [204]:
expected = (1/np.sqrt(2) * np.sqrt(0.95) - 1/np.sqrt(2) * np.sqrt(0.05))**2
expected

0.2820550528229661

In [205]:
direct_sum_result = benchmark_certification_using_direct_sum(
    backend=simulator,
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_v1_direct_sum_dag=v0_v1_direct_sum_dag(),
    num_shots_per_measurement=100000
)

direct_sum_result

0.27924

In [206]:
p_succ = expected
print(f"Analytical p_succ = {p_succ}")
print(
f"Postselection: p_succ = {postselection_result}, abs_error ={np.abs(p_succ - postselection_result)}"
)
print(f"Direct sum: p_succ = {direct_sum_result}, abs_error ={np.abs(p_succ - direct_sum_result)}")

Analytical p_succ = 0.2820550528229661
Postselection: p_succ = 0.7142857142857143, abs_error =0.4322306614627482
Direct sum: p_succ = 0.27924, abs_error =0.0028150528229661242


In [207]:
from qbench.schemes.postselection import (
    assemble_circuits_discrimination_postselection,
    compute_probabilities_certification_postselection,
)
circuits = assemble_circuits_discrimination_postselection(
target=0,
ancilla=1,
state_preparation=state_prep(),
u_dag=u_dag(),
v0_dag=v0_dag(),
v1_dag=v1_dag(),
)

In [208]:
from qiskit_aer.noise import NoiseModel, ReadoutError
error = ReadoutError([[0.75, 0.25], [0.8, 0.2]])
noise_model = NoiseModel()
noise_model.add_readout_error(error, [0])
noise_model.add_readout_error(error, [1])

In [209]:
keys_ordering = ["u_v0", "u_v1"]

all_circuits = [circuits[key] for key in keys_ordering]

sampler_noiseless = SamplerV2()
sampler_noisy = SamplerV2(options=dict(backend_options=dict(noise_model=noise_model)))

counts_noisy = [sampler_noisy.run(
    [circ],
    shots=100000).result()[0].data.meas.get_counts() for circ in all_circuits]

counts_noiseless = [sampler_noiseless.run(
    [circ],
    shots=100000).result()[0].data.meas.get_counts() for circ in all_circuits]

In [210]:
prob_succ_noiseless = compute_probabilities_certification_postselection(
    u_v0_counts=counts_noiseless[0],
    u_v1_counts=counts_noiseless[1],)

prob_succ_noisy = compute_probabilities_certification_postselection(
    u_v0_counts=counts_noisy[0],
    u_v1_counts=counts_noisy[1],)

In [211]:
print(prob_succ_noiseless)

0.2826338159785372


In [212]:
print(prob_succ_noisy)

0.7728359732242023
